In [1]:
import timm
import torch
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TRANSFORM = transforms.ToTensor()

In [3]:
class TIMM:
    def __init__(self):
        # https://huggingface.co/timm/vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k
        self.model = timm.create_model(
            'vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
        self.model.eval()

        # get model specific transforms (normalization, resize)
        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=False)

    @functools.lru_cache(maxsize=None)
    def extract(self, img) -> torch.Tensor:
        with torch.no_grad():
            return self.model(self.transforms(img).unsqueeze(0))[0]

timm_model = TIMM()

In [4]:
# transforma cada par em uma linha de uma ndarray (é quem criou os CSVs dev_train e dev_val)
def pairs_as_ndarray(pairs):
    result = []
    for i, pair in enumerate(pairs):
        image1, image2, label = pair
        print(f'Progress: {i}/{len(pairs)}         ', end='\r')
        attr1 = timm_model.extract(image1)
        attr2 = timm_model.extract(image2)
        row = np.concatenate((attr1, attr2, [label]))
        result.append(row)
    return np.array(result)

def duplicate_by_symmetry(x):
    result = []
    for row in x:
        num_of_features = int((len(row) - 1) / 2)
        left = row[:num_of_features]
        right = row[num_of_features:-1]
        label = row[-1]
        result.append(np.concatenate((left, right, [label])))
        result.append(np.concatenate((right, left, [label])))
    return np.array(result)

In [5]:
# dev_train = pairs_as_ndarray(Preprocess.load_train_pairs(transform=TRANSFORM))
# dev_val = pairs_as_ndarray(Preprocess.load_test_pairs(transform=TRANSFORM))
# Load from .npy
dev_train = duplicate_by_symmetry(np.load('dev_train.npy'))
dev_val = np.load('dev_val.npy')
dev_train.shape, dev_val.shape

((4400, 1025), (1000, 1025))

In [6]:
# # Save the arrays
# np.save('dev_train.npy', dev_train)
# np.save('dev_val.npy', dev_val)

In [7]:
X_train = dev_train[:, :-1]
y_train = dev_train[:, -1]
X_val = dev_val[:, :-1]
y_val = dev_val[:, -1]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((4400, 1024), (4400,), (1000, 1024), (1000,))

In [8]:
### Regressão logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 0.0031622776601683794, 'solver': 'newton-cg'}
Best cross-validation score:  0.5972727272727273


In [9]:

logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))


Train accuracy: 0.73
Val accuracy: 0.604


In [10]:

### Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)


Best parameters:  {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 30}
Best cross-validation score:  0.7154545454545456


In [11]:

rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))


Train accuracy: 0.9995454545454545
Val accuracy: 0.743


In [12]:

### Rede neural
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs2',
    project_name=f'nn-timm-mediumd')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]


Trial 5 Complete [00h 00m 34s]
val_accuracy: 0.7623999953269959

Best val_accuracy So Far: 0.8033999919891357
Total elapsed time: 00h 02m 36s
Results summary
Results in runs2\nn-timm-mediumd
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
num_layers: 1
units_0: 8
learning_rate: 0.01
units_1: 24
Score: 0.8033999919891357

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 8
learning_rate: 0.01
units_1: 4
Score: 0.7893999934196472

Trial 4 summary
Hyperparameters:
num_layers: 3
units_0: 4
learning_rate: 0.01
units_1: 32
units_2: 4
Score: 0.7623999953269959

Trial 2 summary
Hyperparameters:
num_layers: 1
units_0: 4
learning_rate: 0.01
units_1: 16
Score: 0.7611999869346618

Trial 1 summary
Hyperparameters:
num_layers: 1
units_0: 4
learning_rate: 0.001
units_1: 32
Score: 0.7508000016212464


In [13]:

best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()


Epoch 1/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6390 - loss: 0.7869 - val_accuracy: 0.7610 - val_loss: 0.4912
Epoch 2/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8271 - loss: 0.3916 - val_accuracy: 0.7630 - val_loss: 0.5077
Epoch 3/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.8552 - loss: 0.3354 - val_accuracy: 0.7960 - val_loss: 0.4605
Epoch 4/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.9017 - loss: 0.2520 - val_accuracy: 0.7990 - val_loss: 0.4894
Epoch 5/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.9155 - loss: 0.2175 - val_accuracy: 0.7780 - val_loss: 0.6269
Epoch 6/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - accuracy: 0.9212 - loss: 0.1982 - val_accuracy: 0.7780 - val_loss: 0.6711
Epoch 7/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.9402 - loss: 0.1565 - val_accuracy: 0.7870 - val_loss: 0.7043
Epoch 8/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.9529 - loss: 0.1237 - va

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 8)              │         8,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,629 (96.21 KB)

 Trainable params: 8,209 (32.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,420 (64.14 KB)

In [14]:

train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.9999 - loss: 2.5242e-04
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.7963 - loss: 1.7515


(0.9993181824684143, 0.8019999861717224)

In [15]:


### SVM
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score:  0.8


In [16]:

# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 1.0
Val accuracy: 0.816
